# Kernel Tuner Tutorial

Welcome to this hands-on Kernel Tuner tutorial! Let's get started with making sure you have everything we need installed. Before you run this, make sure you have a working CUDA installation.

If you have not yet installed Kernel Tuner you may run the following cell by selecting it and pressing Shift+Enter. If you have already installed Kernel Tuner then feel free to skip this cell.

In [ ]:
%pip install numpy
%pip install kernel_tuner[pycuda]

If needed, you can restart the kernel from the menu at the top of the notebook. Once we have everything installed we can import the modules that we will need for this tutorial.

In [ ]:
import numpy as np
import kernel_tuner

Our first GPU kernel will be stored in a file that is created by running the following cell:

In [ ]:
%%writefile kernel.cu

__global__ void vector_add(float *c, float *a, float *b, int n) {
    int i = blockIdx.x * block_size_x + threadIdx.x;
    if (i<n) {
        c[i] = a[i] + b[i];
    }
}

This kernel is a very basic vector addition kernel that computes a point-wise addition of vectors A and B and stores the result in vector C. n denotes the length of the vectors. "block_size_x" is not yet defined in this code. We will use Kernel Tuner to run this kernel with different numbers of threads per block and see which kernel configuration is the fastest.

In case you'd like to check the contents of our newly created kernel file from this notebook you can run the following cell:

In [ ]:
%pycat kernel.cu

Now that we have our kernel source code in a file, we can get started on the tuning script! We start by generating some input data that we will use to benchmark our GPU kernel.

In [ ]:
# The size of our vectors that we pass to our vector add kernel
size = 1000000

# all the kernel input and output data need to use numpy data types,
# note that we explicitly state that these arrays should consist of
# 32 bit floating-point values, to match our kernel source code
a = np.random.randn(size).astype(np.float32)
b = np.random.randn(size).astype(np.float32)
c = np.zeros_like(b)
n = np.int32(size)

# Now we combine these variables in an argument list, which matches
# the order and types of the function arguments of our GPU kernel
args = [c, a, b, n]

# The next step is to create a dictionary to tell Kernel Tuner about
# the tunable parameters in our code and what values these may take
tune_params = dict()
tune_params["block_size_x"] = [32, 64, 128, 256, 512]

# Finally, we call tune_kernel to start the tuning process. To do so,
# we pass 
#    the name of the kernel we'd like to tune, in our case: "vector_add", 
#    the name of the file containing our source code,
#    the problem_size that our kernel operates on
#    the argument list to call our kernel function
#    the dictionary with tunable parameters
res, env = kernel_tuner.tune_kernel("vector_add", "kernel.cu", size, args, tune_params)

You can see that tune_kernel printed the run time of the different kernel configurations in our search space.
tune_kernel also returned two outputs that we saved as res and env:
* res is a list of dictionaries, each containing detailed information about the configurations that have been benchmarked
* env is a dictionary that stores information about the hardware and software environment in which this experiment took place. It is recommended to store this information along with the benchmark results. 

You can inspect the output if you'd like:

In [ ]:
print(res[0])

As you can see the run time that tune_kernel had printed is actually the average of 7 individual runs. You can change this default value to any number N by passing the options ``iterations=N`` to tune_kernel.

## Step 2. Problem sizes and thread block dimensions

You may have noticed that in the vector_add example we don't really specify the number of threads per block and the total number of thread blocks that Kernel Tuner should use to tune our kernel. These are inferred automatically by the tuner from the problem_size which we did specify and the use of a tunable parameter with the special name "block_size_x" which Kernel Tuner interprets as the thread block x-dimension.

This is a good point to answer some questions that may arise. 

* What happens if we have multiple dimensions?

The single dimension behavior directly translates to multiple dimensions. You can pass a tuple as the problem size to specify the number of items in each dimension and use block_size x, y, and z to denote the thread block dimensions in up to three dimensions.

* How do I specify the number of thread blocks for kernels that don't follow this pattern?

In Kernel Tuner we try to provide good defaults for the most common use-cases, but also offer the flexibility to overwrite those defaults and specify what you need. In cases where the problem_size is divided by more than just the thread block dimensions, for example if multiple elements are processed by each thread, you can tell Kernel Tuner about it by specifying a grid divisor list. These are lists containing the names of tunable parameters that all divide the problem_size in that dimension to calculate the number of thread blocks. Note that the result of this division is ceiled to the nearest integer.

If you prefer to specify the number of thread blocks directly, rather than through a division of the problem size, perhaps you are using a grid-strided loop (see Reduction example), you can tell Kernel Tuner by setting problem_size directly to the desired number of thread blocks and passing empty lists as grid_divisor lists. This may seem a bit cumbersome, but remember that the user interface is optimized for the most common use cases. It's good to remember that problem_size does not need to be a tuple of integers, you may also use strings to specify the names of any tunable parameters in case you'd like to tune the number of thread blocks that execute the kernel.

* Can I use a name different from "block_size_x"?

Of course, please use tune_kernel's optional argument "block_size_names" to specify the names of the tunable parameters that you use for the thread block dimensions.



Let's do a little exercise with what we've just learned about multiple dimensions and grid divisor lists. First, we should define a kernel that we want to tune:

In [ ]:
%%writefile kernel2d.cu

__global__ void vector_add(float *c, float *a, float *b, int n, int m) {
    int i = blockIdx.x * block_size_x + threadIdx.x;
    int j = blockIdx.y * block_size_y + threadIdx.y;
    if ((i<n) && (j<m)) {
        c[j*n+i] = a[j*n+i] + b[j*n+i];
    }
}

This kernel is almost the same as our previous vector_add kernel, only the vectors are now assumed to be two-dimensional arrays of size n by m. We still do an element-wise addition of vectors a and b, and store the result in c.

In [ ]:
# We again start with preparing the input and output data of our kernel
n = np.int32(1e4)
m = np.int32(1e4)
a = np.random.randn(n*m).astype(np.float32)
b = np.random.randn(n*m).astype(np.float32)
c = np.zeros_like(b)

# And combine these into an argument list that fits our kernel
args = [c, a, b, n, m]

# Now we have to define the two tunable parameters in our code and what values they may take
# Let's just pick a few sensible options
tune_params = dict()
tune_params["block_size_x"] = [32, 64, 128, 256, 512]
tune_params["block_size_y"] = [1, 2, 4, 8]

# Now we are ready to call tune kernel again, but this time for our 2-dimensional problem
res, _ = kernel_tuner.tune_kernel("vector_add", "kernel2d.cu", (n, m), args, tune_params)

You may notice that Kernel Tuner has silently skipped a bunch of configurations! What about 256x8?! What about 512x4 and 512x8?! Kernel Tuner actually queries the device to detect the maximum number of threads before benchmarking and silently skips over configurations that can't be executed. It does that too for configurations that cannot be compiled because they use too much shared memory, or kernels that cannot be launched at runtime for using too many registers. You can modify the above call to tune_kernel by adding the option ``verbose=True``, to tell Kernel Tuner that you want to hear about skipped configurations. Feel free to give that a try!

Let's modify our kernel a bit further. We introduce another tunable parameter, which allows us to vary the amount of work performed by each thread in the x-dimension. Note that when threads are processing more than one element in a particular dimension, we either need to create fewer threads or use fewer thread blocks. We use an implementation that will use fewer thread blocks when increasing the work per thread.

In [ ]:
%%writefile kernel2dw.cu

__global__ void vector_add(float *c, float *a, float *b, int n, int m) {
    int i = blockIdx.x * block_size_x*work_per_thread_x + threadIdx.x;
    int j = blockIdx.y * block_size_y + threadIdx.y;
    for (int ti=0; ti<work_per_thread_x; ti++) {
        if (((i+ti*block_size_x)<n) && (j<m)) {
            c[j*n+(i+ti*block_size_x)] = a[j*n+(i+ti*block_size_x)] + b[j*n+(i+ti*block_size_x)];
        }
    }
}

In [ ]:
# Now we have to add our new tunable parameter work_per_thread_x to our dictionary of tunable parameters.
# We'll keep the number of possible values extremely low for this tutorial because the total number of
# possible configurations in the search space explodes really quickly.
tune_params["work_per_thread_x"] = [1, 2]

# If we were to call tune_kernel now in the same way as we did before it would use too many thread blocks
# for configurations that do more work per thread. Therefore, we have to tell Kernel Tuner that we now have
# another parameter (in addition to the block_size_x) that divides the number of thread blocks in the x dimension.
grid_div_x = ["block_size_x", "work_per_thread_x"]

# Now we are ready to call tune kernel again, but this time using grid_div_x
res, _ = kernel_tuner.tune_kernel("vector_add", "kernel2dw.cu", (n, m), args, tune_params, grid_div_x=grid_div_x)

Now you might be wondering if we can output something more meaningful than the kernel run time. GPU programmers typically use two different metrics to quantify the performance of their kernels. For bandwidth-limited kernels we focus on the achieved throughput in GB/s and for compute-bound kernels we focus on compute performance in GFLOP/s (giga floating-point operations per second). We can calculate these ourselves based on the output returned by tune_kernel, but we can also tell Kernel Tuner to compute user-defined metrics. 

In [ ]:
# Because metrics are composable (meaning that we can define a metric and then use it in the definition of another)
# We have to specify the order in which we define metrics, which we do by using an OrderedDict.
from collections import OrderedDict
metrics = OrderedDict()

# We can specify for example how Kernel Tuner should calculate the GFLOP/s metric of our kernel
# by passing a function that calculates the total number of floating-point operations and dividing
# by 1*10^9 (or 1e9 for short). Time in Kernel Tuner is expressed in miliseconds by default,
# and therefore we divide the measured time by a thousand to convert to time in milliseconds
# to time in seconds.
metrics["GFLOP/s"] = lambda p : (n*m/1e9) / (p["time"]/1000)

# The function defined using a lambda is assumed to receive a dictionary with the benchmark results and
# the tunable parameters used in this specific configuration. Similar to the information Kernel Tuner
# prints to the screen or returns in the results dictionary. Therefore we can access the run time
# of this configuration using the "time" key in the dictionary.

# However for our bandwidth-limited 2D vector add kernel the throughput in GB/s will be more relevant.
# To demonstrate the composability of metrics we first define time_s as the time in seconds
metrics["time_s"] = lambda p : p["time"]/1e3

metrics["GB/s"] = lambda p : (n*m*2*4/1e9) / (p["time_s"])
# We compute the throughput of our kernel as the number of values (n*m) times two (for a and b)
# and times four (for 4 bytes per float) divided by 1e9 for GB, divided by the kernel runtime in seconds

# Now if we run tune_kernel with these metrics we get the following output:
res, _ = kernel_tuner.tune_kernel("vector_add", "kernel2dw.cu", (n, m), args, tune_params,
                                  grid_div_x=grid_div_x, metrics=metrics)

## Code optimizations

There are many possible code optimizations that can be applied to GPU kernels. One common code optimization is partial loop unrolling. Loop unrolling is the process of merging subsequent iterations of a loop into single iterations and reduce the total number of iterations executed by the loop. The idea is that unrolling allows to reduce instruction overhead on evaluating the conditional in the loop and increasing instruction-level parallelism. So now we will have a look on how to tune a loop unrolling factor with Kernel Tuner.

To auto-tune a partial loop unrolling factor in CUDA, Kernel Tuner needs to do a bit more work than usual. As a loop unrolling factor specified in the ``#pragma unroll`` can only be set using a constant integer expression, and a factor of 0 (meaning let the compiler decide whether and how to unroll this loop) is not allowed. Note that a unrolling factor of 1 means telling the compiler to not unroll the loop. Kernel Tuner recognizes this tunable parameter as a partial loop unrolling factor because the name of the tunable parameter starts with "loop_unroll_factor".

First we have to modify our kernel:

In [ ]:
%%writefile kernel2dwu.cu

__global__ void vector_add(float *c, float *a, float *b, int n, int m) {
    int i = blockIdx.x * block_size_x*work_per_thread_x + threadIdx.x;
    int j = blockIdx.y * block_size_y + threadIdx.y;
    #pragma unroll loop_unroll_factor_0
    for (int ti=0; ti<work_per_thread_x; ti++) {
        if (((i+ti*block_size_x)<n) && (j<m)) {
            c[j*n+(i+ti*block_size_x)] = a[j*n+(i+ti*block_size_x)] + b[j*n+(i+ti*block_size_x)];
        }
    }
}

In [ ]:
# And we modify our tunable parameters to include our loop unrolling factor
# We also reduce the number of possible block sizes to keep the tuning interactive

tune_params = dict()
tune_params["block_size_x"] = [32, 64]
tune_params["block_size_y"] = [1, 2]
tune_params["work_per_thread_x"] = [1, 2, 4, 8, 16]
tune_params["loop_unroll_factor_0"] = [0, 1, 2, 4]

# The tunable parameters now allow the tuner to impose a loop unrolling factor that may
# be higher than the actual number of iterations of our loop. In these situations
# the loop unrolling won't have any effect, so we can skip these configurations.

# We can tell Kernel Tuner to skip configurations by specifying restrictions
restrictions = ["work_per_thread_x >= loop_unroll_factor_0"]
# restrictions is a list of expressions that should all evaluate to True for 
# a configuration to be considered part of the search space

# And we call tune_kernel again, but now with our newly-defined tunable parameters and restrictions
res, _ = kernel_tuner.tune_kernel("vector_add", "kernel2dwu.cu", (n, m), args, tune_params,
                                  grid_div_x=grid_div_x, metrics=metrics, restrictions=restrictions)

Now the results might differ per GPU, but when developing this notebook on a Nvidia V100 the performance increased by almost factor of 2.

## Using a search optimizing strategy

You may have noticed that we intentionally kept the number of possible values of our tunable parameters to a minimum to reduce the time spent tuning. Clearly this pruning may cut out parts of the search space that contain highly-efficient configurations that we do not know about. 

Therefore, Kernel Tuner supports a large range of optimization strategies, which we will try out now. First, we redefine our tunable parameters, with a much larger number of possible values for all parameters.

In [ ]:
tune_params = OrderedDict()
tune_params["block_size_x"] = [32*i for i in range(1,17)]
tune_params["block_size_y"] = [2**i for i in range(6)]
tune_params["work_per_thread_x"] = [2**i for i in range(6)]
tune_params["loop_unroll_factor_0"] = [0]+[2**i for i in range(6)]

print(tune_params)

In [ ]:
# The total number of possible configurations before applying restrictions in our search space is:
np.prod([len(v) for k, v in tune_params.items()])

In [ ]:
# Now we will call tune_kernel again, but with a random sampler that samples only 2% of our search space
res, _ = kernel_tuner.tune_kernel("vector_add", "kernel2dwu.cu", (n, m), args, tune_params,
                                  grid_div_x=grid_div_x, metrics=metrics, restrictions=restrictions,
                                  strategy="genetic_algorithm", strategy_options=dict(popsize=10, maxiter=10))

In [ ]:
# Kernel Tuner still returns all the information about all benchmarked configurations
# so we can look at res to see how many configurations have been benchmarked
len(res)

So our optimization algorithm only evaluated len(res) configurations, while the search space was much larger. Many of the search strategies in Kernel Tuner are stochastic however, so there are no guarantees it will always find the global optimum.

We close with another very useful feature that will allow Kernel Tuner to continue from an earlier auto-tuning measurement in case something went wrong. For this we can use the ``cache=filename`` option. Kernel Tuner will create a cachefile with the specified filename and store all benchmarked configurations in that file. If you continue from an existing cachefile Kernel Tuner will skip all configurations that have already been benchmarked.

In [ ]:
# Now we will call tune_kernel again, but with a random sampler that samples only 2% of our search space
res, _ = kernel_tuner.tune_kernel("vector_add", "kernel2dwu.cu", (n, m), args, tune_params,
                                  grid_div_x=grid_div_x, metrics=metrics, restrictions=restrictions,
                                  strategy="genetic_algorithm", strategy_options=dict(popsize=10, maxiter=10),
                                 cache="my_test_cache_file")

In [ ]:
# You can inspect the contents of the cachefile with by running this cell
%pycat my_test_cache_file.json

That's all for now! For more tutorials, examples, guides, and information on Kernel Tuner, please see these pages:

Documentation
https://benvanwerkhoven.github.io/kernel_tuner/index.html

Github repository
https://github.com/benvanwerkhoven/kernel_tuner